# Install pyrecdp from github

In [ ]:
!pip install 'git+https://github.com/intel/e2eAIOK.git#egg=pyrecdp&subdirectory=RecDP'

# Install jdk for pyspark running

In [ ]:
!DEBIAN_FRONTEND=noninteractive apt-get install -y openjdk-8-jre

# Prepare test data

In [ ]:
%mkdir -p /content/test_data
%cd /content/test_data
!wget https://raw.githubusercontent.com/intel/e2eAIOK/main/RecDP/tests/data/llm_data/arxiv_sample_100.jsonl

# Import toxicity score function

In [2]:
from pyrecdp.primitives.llmutils import toxicity_score_spark, toxicity_score
from pyrecdp.core import SparkDataProcessor

JAVA_HOME is not set, use default value of /usr/lib/jvm/java-8-openjdk-amd64/


/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


# Specify variables

In [4]:
data_file = '/content/test_data/arxiv_sample_100.jsonl'
save_path = '/content/test_data/output/toxicity_score'

# Load data

In [6]:
rdp = SparkDataProcessor()
spark = rdp.spark
spark_df = spark.read.json(data_file)
spark_df.show()

Will assign 1 cores and 10386 M memory for spark
per core memory size is 10.143 GB and shuffle_disk maximum capacity is 8589934592.000 GB
+--------------------+--------------------+
|                meta|                text|
+--------------------+--------------------+
|{2203.15369, en, ...|\section{Introduc...|
|{math/9807097, en...|\section{Introduc...|
|{2008.06948, en, ...|\section{Introduc...|
|{cond-mat/9807071...|\section{Introduc...|
|{2210.10650, en, ...|\section{\label{s...|
|{astro-ph/9807119...|\section{Introduc...|
|{2111.03152, en, ...|\section{Introduc...|
|{1606.04992, en, ...|\n\n\section{Intr...|
|{1608.03404, en, ...|\section{introduc...|
|{1904.10101, en, ...|\section{Introduc...|
|{cond-mat/9807275...|\section{Introduc...|
|{2109.05334, en, ...|\section{Introduc...|
|{1512.06966, en, ...|\section{Introduc...|
|{2112.04926, en, ...|\section{Introduc...|
|{2202.01000, en, ...|\section{Introduc...|
|{2209.13421, en, ...|\section{Introduc...|
|{1103.5603, en, 2...|\sec

# Process the 'text' column and generate toxicity score which will determine whether the row kept or not(spark dataframe interface)

In [7]:
toxicity_score_spark_df = toxicity_score_spark(spark_df)
toxicity_score_spark_df.show()

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.4-alpha/multilingual_debiased-0b549669.ckpt" to /root/.cache/torch/hub/checkpoints/multilingual_debiased-0b549669.ckpt
100%|██████████| 1.04G/1.04G [00:04<00:00, 252MB/s]


+--------------------+--------------------+--------------------+
|                meta|                text|       text_toxicity|
+--------------------+--------------------+--------------------+
|{2203.15369, en, ...|\section{Introduc...|2.222859766334295...|
|{math/9807097, en...|\section{Introduc...|2.392058377154171...|
|{2008.06948, en, ...|\section{Introduc...|3.047155332751572E-4|
|{cond-mat/9807071...|\section{Introduc...|2.939336118288338E-4|
|{2210.10650, en, ...|\section{\label{s...|2.322625950910151E-4|
|{astro-ph/9807119...|\section{Introduc...|2.402916288701817...|
|{2111.03152, en, ...|\section{Introduc...|2.399834484094753...|
|{1606.04992, en, ...|\n\n\section{Intr...|2.001557877520099...|
|{1608.03404, en, ...|\section{introduc...|1.896593603305518...|
|{1904.10101, en, ...|\section{Introduc...|3.666119009722024E-4|
|{cond-mat/9807275...|\section{Introduc...|2.205321216024458...|
|{2109.05334, en, ...|\section{Introduc...|1.802597107598558E-4|
|{1512.06966, en, ...|\se

# Process the 'text' column and generate toxicity score and save data as parquet format.

In [5]:
toxicity_score(data_file, save_path, data_file_type="jsonl")
!ls $save_path

init ray
init ray with total mem of 8167961395, total core of 1


2023-10-17 13:41:04,936	WARNING services.py:1889 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 6133121024 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=8.37gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-10-17 13:41:06,195	INFO worker.py:1642 -- Started a local Ray instance.


execute with ray started ...


2023-10-17 13:41:07,953	INFO read_api.py:406 -- To satisfy the requested parallelism of 5, each read task output is split into 5 smaller blocks.
2023-10-17 13:41:13,137	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadText->SplitBlocks(5)] -> TaskPoolMapOperator[Map(convert_json)->Map(<lambda>)->Write]
2023-10-17 13:41:13,140	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
2023-10-17 13:41:13,153	INFO streaming_executor.py:96 -- Tip: For detailed progress reporting, run `ray.data.DataContext.get_current().execution_options.verbose_progress = True`


Running 0:   0%|          | 0/25 [00:00<?, ?it/s]

(Map(convert_json)->Map(<lambda>)->Write pid=3321) 2023-10-17 13:41:28.868788: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-17 13:44:18,638	WARNING plan.py:567 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more CPUs are freed up. A common reason is that cluster resources are used by Actors or Tune trials; see the following link for more details: https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune
2023-10-17 13:44:18,661	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadText->SplitBlocks(5)] -> TaskPoolMapOperator[Map(convert_json)->Map(<lambda>)]
2023-10-17 13:44:18,664	INFO streaming_executor.py:94 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbo

Running 0:   0%|          | 0/25 [00:00<?, ?it/s]

2023-10-17 13:45:12,597	WARNING worker.py:2058 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: 4025912f2c88b1553b235d1a53b4d7942176c9df01000000 Worker ID: acf44c4a3aabd19b4abc0c5620f6a5d5b49fbd41da89d802d2b9db97 Node ID: ee5521b085cc011722630c1c90a9f41b0f42a13ad2c34444b89e4dfb Worker IP address: 172.28.0.12 Worker port: 32929 Worker PID: 3321 Worker exit type: SYSTEM_ERROR Worker exit detail: The leased worker has unrecoverable failure. Worker is requested to be destroyed when it is returned.
(Map(convert_json)->Map(<lambda>) pid=4454) 2023-10-17 13:45:30.356916: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


execute with ray took 403.94510068599993 sec
b63fe36529b64d6c8bcc397fdc69cb5f_000000_000000.parquet
b63fe36529b64d6c8bcc397fdc69cb5f_000001_000000.parquet
b63fe36529b64d6c8bcc397fdc69cb5f_000002_000000.parquet
b63fe36529b64d6c8bcc397fdc69cb5f_000003_000000.parquet
b63fe36529b64d6c8bcc397fdc69cb5f_000004_000000.parquet
